In [ ]:
import os
import cv2
import numpy as np
import pickle
from tqdm import tqdm
import mediapipe as mp

dataset_path = r'C:\Users\moham\Downloads\Robotech\Summer_Training\Project\Note_Book\Train_data'

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)

data = []
labels = []
label_dict = {}  

all_folders = sorted(os.listdir(dataset_path))

for idx, folder in enumerate(all_folders):
    label_dict[idx] = folder.upper()

for idx, folder in enumerate(all_folders):
    
    folder_path = os.path.join(dataset_path, folder)
    label = folder.upper()

    for file in tqdm(os.listdir(folder_path), desc=f"Processing '{label}'"):
        
        img_path = os.path.join(folder_path, file)
        img = cv2.imread(img_path)

        if img is None:
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:

            for hand_landmarks in results.multi_hand_landmarks:
                x_ = []
                y_ = []
                features = []

                for lm in hand_landmarks.landmark:
                    x_.append(lm.x)
                    y_.append(lm.y)

                for lm in hand_landmarks.landmark:
                    features.append(lm.x - min(x_))
                    features.append(lm.y - min(y_))

                if len(features) == 42:
                    data.append(features)
                    labels.append(idx)

data = np.array(data)
labels = np.array(labels)

Processing 'Z': 100%|██████████| 900/900 [00:25<00:00, 35.95it/s]


In [ ]:
from tensorflow.keras.utils import to_categorical

num_classes = len(label_dict)
labels_cat = to_categorical(labels, num_classes=num_classes)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, labels_cat, test_size=0.2, random_state=42, stratify=labels
)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(128, activation='relu', input_shape=(42,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\moham\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',  
    patience=5,         
    restore_best_weights=True 
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=300,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/300
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2930 - loss: 2.3276 - val_accuracy: 0.9036 - val_loss: 0.4636
Epoch 2/300
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8557 - loss: 0.4905 - val_accuracy: 0.9633 - val_loss: 0.2097
Epoch 3/300
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9213 - loss: 0.2767 - val_accuracy: 0.9761 - val_loss: 0.1501
Epoch 4/300
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9444 - loss: 0.2178 - val_accuracy: 0.9674 - val_loss: 0.1473
Epoch 5/300
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9610 - loss: 0.1676 - val_accuracy: 0.9780 - val_loss: 0.1094
Epoch 6/300
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9655 - loss: 0.1510 - val_accuracy: 0.9805 - val_loss: 0.0943
Epoch 7/300
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9637 - loss: 0.1483 - val_accuracy: 0.9821 - val_loss: 0.0939
Epoch 8/300
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9703 - loss: 0.1319 - val_accu

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9882 - loss: 0.0661  
Test Accuracy: 0.99


In [ ]:
from tensorflow.keras.models import load_model
import pickle

model.save("asl_mlp_model.h5")

In [ ]:
lab={}
all_folders = sorted(os.listdir(dataset_path))

for idx, folder in enumerate(all_folders):
    lab[idx] = folder.upper()

In [ ]:
with open("label_dict.pkl", "wb") as f:
    pickle.dump(lab, f)